In [5]:
import tensorflow as tf
import numpy as np
import pandas as pd
from datetime import datetime
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import InputLayer, Input, Dense, Activation, Conv2D, MaxPooling2D, Flatten, Concatenate
from keras.utils.np_utils import to_categorical
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.models import load_model

Conv2D_1_Filters = 64
Conv2D_1_KernelSize = (17, 17)
Conv2D_1_Activation = "relu"

Conv2D_2_Filters = 64
Conv2D_2_KernelSize = (17, 17)
Conv2D_2_Activation = "relu"

Conv2D_3_Filters = 64
Conv2D_3_KernelSize = (17, 17)
Conv2D_3_Activation = "relu"

MaxPool_Size = (2, 2)

Dense1_Units = 512
Dense1_Activation = "relu"

Dense2_Units = 10
Dense2_Activation = "softmax"

LearningRate = 0.01
Decay = 0.00001
Momentum = 0.9

Epochs = 24
BatchSize = 256
ValidationSplit = 0.2

params = [f"Conv2D_1: {Conv2D_1_Filters}, {Conv2D_1_KernelSize}, {Conv2D_1_Activation}",
          f"Conv2D_1: {Conv2D_2_Filters}, {Conv2D_2_KernelSize}, {Conv2D_2_Activation}",
          f"Conv2D_1: {Conv2D_3_Filters}, {Conv2D_3_KernelSize}, {Conv2D_3_Activation}",
          f"MaxPooling2D: {MaxPool_Size}",
          f"Concatenate - Conv2D_1, Conv2D_2, Conv2D_3, MaxPool",
          f"Flatten - No Params",
          f"Dense1: {Dense1_Units}, {Dense1_Activation}",
          f"Dense2: {Dense2_Units}, {Dense2_Activation}",
          f"Learning Rate: {LearningRate}",
          f"Decay: {Decay}",
          f"Momentum: {Momentum}",
          f"Epochs: {Epochs}",
          f"Batch Size: {BatchSize}",
          f"Validation Split: {ValidationSplit}"]

(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()

y_train = to_categorical(y_train, num_classes=10)
y_test = to_categorical(y_test, num_classes=10)

#x_train = np.reshape(x_train,(50000,32*32*3))
#x_test = np.reshape(x_test,(10000,32*32*3))

x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

input_layer = Input(shape=(32,32,3), name="input")
convLayer1 = Conv2D(Conv2D_1_Filters, Conv2D_1_KernelSize, activation=Conv2D_1_Activation)(input_layer)
convLayer2 = Conv2D(Conv2D_2_Filters, Conv2D_2_KernelSize, activation=Conv2D_2_Activation)(input_layer)
convLayer3 = Conv2D(Conv2D_3_Filters, Conv2D_3_KernelSize, activation=Conv2D_3_Activation)(input_layer)
maxPoolLayer = MaxPooling2D(pool_size=MaxPool_Size)(input_layer)
concatenateLayer = Concatenate()([convLayer1, convLayer2, convLayer3, maxPoolLayer])
flattenLayer = Flatten()(concatenateLayer)
denseLayer1 = Dense(Dense1_Units, activation=Dense1_Activation)(flattenLayer)
denseLayer2 = Dense(Dense2_Units, activation=Dense2_Activation, name="output")(denseLayer1)
model = tf.keras.Model(inputs=[input_layer], outputs=[denseLayer2])

mySGD = SGD(learning_rate=LearningRate, decay=Decay, momentum=Momentum)
model.compile(optimizer=mySGD, loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()

history = model.fit({"input": x_train}, {"output": y_train}, epochs=Epochs, batch_size=BatchSize, validation_split=ValidationSplit)

training_score = model.evaluate(x_train, y_train, verbose=0)
test_score = model.evaluate(x_test, y_test, verbose=0)

print(training_score)
print(test_score)

currentTime = datetime.now()
timeStr = currentTime.strftime("%m-%d-%Y_%H:%M:%S")

logFile = open(f"Model3_History_{timeStr}.txt", "w+")

logFile.write("Parameters:\n")
for s in params:
    logFile.write(f"{s}\n")
logFile.write("=================================================\n")

for e in range(Epochs):
    logFile.write(f"Epoch: {e+1}/{Epochs} - ")
    for key in history.history.keys():
        logFile.write(f"{key}: {round(history.history[key][e], 4)} - ")
    logFile.write("\n")
    
logFile.write("=================================================\n")
logFile.write("[Loss, Accuracy]\n")
logFile.write(f"Training Score = {training_score}\n")
logFile.write(f"Test Score = {test_score}\n")
logFile.close()
model.save(f"Model3_{timeStr}.h5")

Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input (InputLayer)              [(None, 32, 32, 3)]  0                                            
__________________________________________________________________________________________________
conv2d_10 (Conv2D)              (None, 16, 16, 64)   55552       input[0][0]                      
__________________________________________________________________________________________________
conv2d_11 (Conv2D)              (None, 16, 16, 64)   55552       input[0][0]                      
__________________________________________________________________________________________________
conv2d_12 (Conv2D)              (None, 16, 16, 64)   55552       input[0][0]                      
____________________________________________________________________________________________

In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
from tensorflow.keras.models import load_model
from keras.utils.np_utils import to_categorical


#Change X_Test and Y_Test for a different data set.
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()

y_test = to_categorical(y_test, num_classes=10)
x_test = np.reshape(x_test,(10000,32*32*3))
x_test = x_test.astype('float32') / 255.0

testModel = load_model('FINAL_MODEL3.h5')
predictions = testModel.predict(x_test)
predictionClass = []
classes = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

for i in range(len(predictions)):
    imagePrediction = np.where(predictions[i] == predictions[i].max())
    predictionClass.append(classes[int(imagePrediction[0])])

print(predictionClass)